In [24]:
import pandas as pd
import os
from pathlib import Path
import re

In [25]:
neighborhood_tracts = {
    'Fenway':['9815.01', '101.03', '101.04', '102.04','104.04', '104.05', '102.03', '104.08', '104.03'],
    'Roxbury':['815', '906','804.01','806.01','9803','814','818','801','820','813','819','817','803','805','821','904'],
    'Back Bay':['106','107.01','107.02','108.01','108.02','105'],
    'South End':['704.02','711.01','712.01','703','705','708','706','707','709']
}

In [26]:
tract_lookup_table = pd.DataFrame(columns = ['Tract','Neighborhood'])
for hood in list(neighborhood_tracts.keys()):
    for tract in neighborhood_tracts[hood]:
        row = {'Tract': tract, 'Neighborhood': hood}
        tract_lookup_table = tract_lookup_table.append(row, ignore_index=True)
tract_lookup = tract_lookup_table.set_index('Tract').to_dict(orient='index')

In [27]:
def lookup(tract):
    tract_no = tract.split( )[2]
    return tract_lookup[tract_no]['Neighborhood']

In [28]:
def get_paths(subset):
    tracts = []
    paths = []
    for path, subdirs, files in os.walk(Path(f'data/{subset}')):
        tracts = subdirs
        for name in files:
            file_path = os.path.join(path,name)
            if name[0] != '.':
                paths.append(file_path)
    return paths

In [29]:
paths = get_paths('Mass Ave Census Tracts')
paths[0]

'data/Mass Ave Census Tracts/708/Household_Income_in_the_Past_12_Months_(In_2019_Inflation-adjusted_Dollars)_5YR_ACS_2015-2019.csv'

In [33]:
def parse(file, bigDF, totalDF):
    df = pd.read_csv(file)
    tract, county, state = (list(df.columns)[1]).split(',')
    #tract_no = tract.split( )[2]
    tract_no = tract
    measure = df.iloc[0][list(df.columns)[0]]
    for i in range(1,df.index.stop-2):
        category = df.iloc[i][list(df.columns)[0]]
        value = df.iloc[i][list(df.columns)[1]]
        row = {
            'Neighborhood': lookup(tract_no),
            'Tract':tract_no,
            'Measure':measure,
            'Category':category,
            'Value':value
        }
        if row['Category'].lower()=='total':
            totalDF = totalDF.append(row, ignore_index=True)
        else:
            bigDF = bigDF.append(row, ignore_index=True)
    return bigDF, totalDF

In [34]:
pd.read_csv(paths[0])

,Tracts,"Census Tract 708, Suffolk County, Massachusetts"
0,Household Income,Count
1,"Less than $14,999",323
2,"$15,000 - $24,999",124
3,"$25,000 - $34,999",60
4,"$35,000 - $49,999",95
5,"$50,000 - $74,999",332
6,"$75,000 - $99,999",110
7,"$99,999 - $149,999",192
8,"$150,000 or more",793
9,Total,2029


In [35]:
def summarize(measure, bigDF, totalDF):
    try:
        os.mkdir(f'data/summaries')
    except:
        pass
    measure = measure.replace('/','-')
    measureDF = bigDF[bigDF['Measure'] == measure].drop(columns=['Measure'])
    measureDF.to_csv(f'data/summaries/{measure}_joined.csv')
    measuretotalDF = totalDF[totalDF['Measure'] == measure].drop(columns=['Measure', 'Category'])
    measuretotalDF.to_csv(f'data/summaries/{measure}_totals.csv')

In [36]:
joinDF = pd.DataFrame(columns=[
    'Neighborhood',
    'Tract',
    'Measure',
    'Category',
    'Value'])
totalDF = joinDF.copy()
paths = get_paths('Mass Ave Census Tracts')
for file in paths:
    joinDF, totalDF = parse(file, joinDF, totalDF)
    joinDF.to_csv(f'data/data_joined.csv')
    totalDF.to_csv(f'data/data_totals.csv')
measures = list(totalDF['Measure'])
for measure in measures:
    summarize(measure, joinDF, totalDF)

In [38]:
totalDF

,Neighborhood,Tract,Measure,Category,Value
0,South End,Census Tract 708,Household Income,Total,2029
1,South End,Census Tract 708,Health Insurance,Total,3597
2,South End,Census Tract 708,Field of Bachelor's Degree for First Major,Total,2386
3,South End,Census Tract 708,Household Size,Total,631
4,South End,Census Tract 708,Year Structure Built,Total,2201
...,...,...,...,...,...
250,Back Bay,Census Tract 107.01,Resident Employment by Industry,Total,1767
251,Back Bay,Census Tract 107.01,Race/Ethnicity,Total,2383
252,Back Bay,Census Tract 107.01,Gender,Total,2383
253,Back Bay,Census Tract 107.01,Age,Total,2383


In [39]:
def convert_numeric(v): return re.sub("[^0-9]", "", v)

In [40]:
joinDF['Value'] = [float(convert_numeric(v)) if isinstance(v, str) else v for v in joinDF['Value']]
grouped = joinDF.drop(columns='Tract').groupby(['Neighborhood','Measure','Category']).sum()
grouped

Value
Neighborhood Measure              Category                    
Back Bay     Age                  0-4                    259.0
                                  18 to 64 years       20262.0
                                  18-24                 1974.0
                                  25-34                 2973.0
                                  35-44                 1079.0
...                                                        ...
South End    Year Structure Built Built 1980 to 1989      90.0
                                  Built 1990 to 1999      71.0
                                  Built 2000 to 2009     148.0
                                  Built 2010 to 2013       8.0
                                  Built 2014 or later      0.0

[616 rows x 1 columns]

In [41]:
""" IGNORE
def get_neighborhood_tracts():
    neighborhoods = []
    tract_dict = {}
    for path, subdirs, files in os.walk(Path('data/Mass Ave Neighborhoods')):
        neighborhoods = subdirs
        break
    for hood in neighborhoods:
        for path, subdirs, files in os.walk(Path('data/Mass Ave Neighborhoods')):
            try:
                tracts = []
                df = pd.read_csv(os.path.join(path,files[0]))
                tract_strings = list(df.columns[2:])
                for t in tract_strings:
                    tract = (t.split(',')[0]).split(' ')[-1]
                    tracts.append(tract)
            except:
                pass
            tract_dict[hood]=list(set(tracts))
    return tract_dict
"""

"\ndef get_neighborhood_tracts():\n    neighborhoods = []\n    tract_dict = {}\n    for path, subdirs, files in os.walk(Path('data/Mass Ave Neighborhoods')):\n        neighborhoods = subdirs\n        break\n    for hood in neighborhoods:\n        for path, subdirs, files in os.walk(Path('data/Mass Ave Neighborhoods')):\n            try:\n                tracts = []\n                df = pd.read_csv(os.path.join(path,files[0]))\n                tract_strings = list(df.columns[2:])\n                for t in tract_strings:\n                    tract = (t.split(',')[0]).split(' ')[-1]\n                    tracts.append(tract)\n            except:\n                pass\n            tract_dict[hood]=list(set(tracts))\n    return tract_dict\n"